In [3]:
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

# Run shell commands only in Colab
if IN_COLAB:
    %pip install beanquery
    %pip install git+https://github.com/Ev2geny/evbeantools.git@develop_pr
    %pip install requests

    print("Packages installed in Colab environment")
else:
    print("Running locally, skipping pip install")

import os
import re
from io import StringIO  
import datetime
from collections.abc import Iterable

import pandas as pd
import numpy as np

import plotly.express as px
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

from beancount.loader import load_file
from beancount.parser import printer
from beancount.parser.printer import print_entries

from beancount.core.data import Transaction, Posting, Open, Close, Balance, Price, Note, Event, Query, Custom

from evbeantools.sing_curr_conv import get_equiv_sing_curr_entries
from evbeantools.juptools import  add_total, beanquery2df,  get_net_worths, get_bean_pivot, prepare_sunburst_data_input, remove_empty_rows, get_period_end_dates, highlight_rows
from evbeantools.beanfuncs import check_mult_funds_in_transit, split_posting
from evbeantools.printer_rich import display_entries

if IN_COLAB:
    import requests

Running locally, skipping pip install


<style>
h1 { color: purple; }
h2 { color: orange; }
</style>

In [5]:
FILE_URL = "https://github.com/beancount/fava/blob/main/contrib/examples/example.beancount"

BEAN_FILE_NAME = "example-fava.beancount"

if IN_COLAB:
    response = requests.get(FILE_URL)

    # Check if the request was successful
    if response.status_code == 200:
        with open(BEAN_FILE_NAME, 'w') as f:
            f.write(response.text)
        print(f"File downloaded and saved as {BEAN_FILE_NAME}")
    else:
        print(f"Failed to download file. Status code: {response.status_code}")



entries, errors, opts = load_file(BEAN_FILE_NAME)
CURR = opts["operating_currency"][0]

# Checks

## Beancount built in checks

In [6]:
printer.print_errors(errors)

# Equivalent entries in singe currency

In [7]:
entries_eq, errors_eq, opts_eq = get_equiv_sing_curr_entries(entries, opts, target_currency=CURR, self_testing_mode=True)

# Chart of accounts

In [8]:
coa_query = """
SELECT account, open.currencies as currencies, open.date as open_date, close.date as close_date 
FROM #accounts
ORDER BY account
"""

coa_df = beanquery2df(entries, opts, coa_query).fillna("")
coa_df.set_index("account", inplace=True)

coa_df_styled = coa_df.style.set_table_styles([
    {'selector': 'th.row_heading',  # Targets the index column
     'props': [('text-align', 'left')]}
])

coa_df_styled

,currencies,open_date,close_date
account,,,
Assets:US:BofA,,2015-01-01,
Assets:US:BofA:Checking,['USD'],2015-01-01,
Assets:US:ETrade:Cash,['USD'],2015-01-01,
Assets:US:ETrade:GLD,['GLD'],2015-01-01,
Assets:US:ETrade:ITOT,['ITOT'],2015-01-01,
Assets:US:ETrade:VEA,['VEA'],2015-01-01,
Assets:US:ETrade:VHT,['VHT'],2015-01-01,
Assets:US:Federal:PreTax401k,['IRAUSD'],1980-05-12,
Assets:US:Hoogle:Vacation,['VACHR'],2015-01-01,


# Financial overview

In [9]:
def get_fin_overview_table(entries, 
                           opt,
                           freq,
                           nw_change_data:dict,
                           *,
                           start_period = None, 
                           qnt_periods = None,
                           end_period = None,
                           currency = None):
    """
    Function to get the financial overview table for the period from start_period to end_period
    
    params:
        entries: list of entries
        freq: str
            pandas period alias (Y, Q, M, W, D)
                https://pandas.pydata.org/docs/user_guide/timeseries.html#timeseries-period-aliases
                
        start_period: Period, str, datetime, date or pandas.Timestamp  . pandas timeperiod representation
                    If not provided, the first date of the first entry will be used to determine the start_period
        
        qnt_periods: int
            number of periods to display
            
        end_period: Period, str, datetime, date or pandas.Timestamp  . pandas timeperiod representation
                    Used only if qnt_periods is not provided. 
                    If not provided, the last date of the last entry will be used to determine the end_period
            
    """
    if not start_period:
        start_period = pd.Period(entries[0].date, freq)
    else:
        start_period = pd.Period(start_period, freq)
    
    if qnt_periods:
        end_period = start_period + qnt_periods - 1
    else:
        if not end_period:
            end_period = pd.Period(entries[-1].date, freq)
        else:
            end_period = pd.Period(end_period, freq)
            
    if not currency:
        currency = opt["operating_currency"][0]        
            
    # This is just to show types        
    start_period: pd.Period = start_period
    end_period: pd.Period = end_period
            
    start_date_iso_str = start_period.start_time.strftime("%Y-%m-%d")
    end_date_iso_str = end_period.end_time.strftime("%Y-%m-%d")
    
    def get_table_line(query, name):
        """Performs standard actions to create a table line

        Returns:
            _type_: _description_
        """
        df = beanquery2df(entries, opts,  query)
        df['period'] = pd.PeriodIndex(df['date'], freq=freq)
        df_pivot = df.pivot_table(values=f'amount ({currency})', 
                                  aggfunc='sum', columns='period')
        df_pivot.index = [name]
        return df_pivot
    
    query_exp=f"""
            select id, date, CONVERT(position,'{currency}', date) as amount, narration 
            WHERE 
               ({nw_change_data["expenses"]["sql"]}) AND 
                date >= {start_date_iso_str} AND 
                date <= {end_date_iso_str} 
            """
    df_expen_pivot_line = get_table_line(query_exp, nw_change_data["expenses"]["name"])
    
    # print(df_expen_pivot_line)
        
    
    query_income = f"""
            select id, date, CONVERT(position,'{currency}',date) as amount, narration 
            WHERE 
                ({nw_change_data["income"]["sql"]})  AND
                date >= {start_date_iso_str} AND 
                date <= {end_date_iso_str} 
            """
            
    df_income_pivot_line = get_table_line(query_income, nw_change_data["income"]["name"])
    
    # print(df_income_pivot_line)
            
    query_gains = f"""
            select id, date, CONVERT(position,'{currency}',date) as amount, narration 
            WHERE 
                ({nw_change_data["gains"]["sql"]}) AND
                date >= {start_date_iso_str} AND 
                date <= {end_date_iso_str} 
            """
    
    df_gains_pivot_line = get_table_line(query_gains, nw_change_data["gains"]["name"])
    
    
    query_equity=f"""
            select id, date, CONVERT(position,'{currency}',date) as amount, narration 
            WHERE 
               ({nw_change_data["equity"]["sql"]}) AND 
                date >= {start_date_iso_str} AND 
                date <= {end_date_iso_str} 
            """
    df_equity_pivot_line = get_table_line(query_equity, nw_change_data["equity"]["name"])
    
    # print(df_equity_pivot_line)
    
    
    df_result = pd.concat([df_expen_pivot_line, df_income_pivot_line, df_gains_pivot_line, df_equity_pivot_line])
    
    df_result = add_total(df_result, col_total_name="TOTAL NW Change")
    
    # for the purposes of NW calculation we need to calculate NW also for the start_period -1
    net_worth_dates: list[datetime.date] = get_period_end_dates(start_period -1 , end_period)
    
    
    
    
    neth_worths:pd.DataFrame = get_net_worths(entries, opts, net_worth_dates, currency, num_acc_components_from_root=1, repeat_row_labels=False)

    # print("---------neth_worths -------------")
    # print(neth_worths)
    # neth_worths.columns = neth_worths.columns.droplevel(0)
    
    # This line converts
    neth_worths = neth_worths.loc[:,f"amount ({currency})"]
    
    # print('-------- neth_worths = neth_worths.loc[:,f"amount ({currency})"] -------------')
    # print(neth_worths)
    
    # neth_worths.set_index('acc_L0', inplace=True)
    neth_worths = add_total(neth_worths, col_total_name="TOTAL NW")
    
    neth_worths.columns = pd.to_datetime(neth_worths.columns).to_period(freq)
    
   
    # print(neth_worths)
    
    # Adding the net worths to the result
    df_result = pd.concat([df_result, neth_worths], axis=0)
    
    # print(df_result)

    df_result = df_result.reindex(sorted(df_result.columns), axis=1)
    
    df_result.fillna(0, inplace=True)
    
    
    df_result.loc["Unexplained diff."] = float('nan')
    
    df_result.loc["Unexplained diff.", df_result.columns[1:]] = (
        df_result.loc["TOTAL NW", df_result.columns[1:]] - df_result.loc["TOTAL NW", df_result.columns[:-1]].values + df_result.loc["TOTAL NW Change", df_result.columns[1:]].values
    )
    
    df_result = highlight_rows(df_result, {"TOTAL NW Change": "lightblue", "TOTAL NW": "lightgreen"} )
    
    
    return df_result

In [10]:

nw_change_data = {"income":{"name": "Income",
                            "sql": "account ~'^Income'  AND not account ~'^Income:Unrealized'"},
                  
                  "gains":{"name": "UnrealGains",
                            "sql": "account ~'^Income:Unrealized'"},
                  
                  "expenses":{"name": "Expenses",
                            "sql": "account ~'^Expenses'"},
                  
                  "equity":{"name": "Equity",
                            "sql": "account ~'^Equity'"}
                  }


fin_overview_table = get_fin_overview_table(entries_eq, opts_eq, "Q", nw_change_data, currency = "USD")

fin_overview_table

,2015Q1,2015Q2,2015Q3,2015Q4,2016Q1,2016Q2,2016Q3,2016Q4,2017Q1,2017Q2,2017Q3
Expenses,"24,218.18","22,567.78","24,781.02","23,715.87","22,711.09","24,788.18","22,307.59","24,375.56","23,198.15","24,611.07","17,334.56"
Income,"-36,677.90","-31,438.20","-33,694.43","-32,570.57","-31,475.56","-36,227.16","-29,702.51","-32,569.35","-32,172.45","-36,779.43","-24,637.02"
UnrealGains,165.47,542.52,-822.58,-780.08,"-1,304.18","-2,413.56","1,338.72",-795.62,"-1,854.50","-5,716.71",817.08
Equity,"-3,490.52",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
TOTAL NW Change,"-15,784.77","-8,327.90","-9,735.99","-9,634.78","-10,068.65","-13,852.54","-6,056.20","-8,989.41","-10,828.80","-17,885.07","-6,485.38"
Assets,"16,138.44","24,419.36","34,492.25","44,462.21","54,234.92","68,583.55","74,760.50","84,219.11","95,086.18","113,451.90","120,352.78"
Liabilities,-353.66,-306.68,-643.54,-978.70,-682.77,"-1,178.89","-1,299.66","-1,768.88","-1,807.13","-2,287.79","-2,703.29"
TOTAL NW,"15,784.78","24,112.68","33,848.71","43,483.51","53,552.15","67,404.66","73,460.84","82,450.23","93,279.05","111,164.11","117,649.49"
Unexplained diff.,nan,-0.00,0.03,0.02,-0.00,-0.03,-0.02,-0.02,0.03,-0.01,-0.00


# Filtering spesific transactions

In [11]:
def check_if_to_filter(entry):    
    file = StringIO("")  
    
    printer.print_entry(entry, file=file)
    file.seek(0)
    entry_str = file.read()

    if "Income:US:ETrade:Dividends" in entry_str:
        return True

filtered_entries = list(filter(check_if_to_filter, entries))

# printer.print_entries(filtered_entries)
display_entries(filtered_entries, write_source=True)

<evbeantools.printer_rich.display_entries.<locals>.DisplayableObject at 0x1bfbff2b350>